In [10]:
%load_ext autoreload
%autoreload 2

import os
import re
import functools
import numpy as np
import pandas as pd
import corescpy as cr

# File Paths
libid = "50452A"
ddd = "/mnt/cho_lab/disk2/elizabeth/data/shared-xenium-library"
dir_data = "/mnt/cho_lab/bbdata2/outputs/TUQ97N"  # data directory
out_dir = os.path.join(ddd, "outputs/TUQ97N/nebraska")  # output directory
file_mdf = os.path.join(ddd, "samples.csv")  # metadata

# Metadata
col_subject, col_condition = "Patient", "Condition"
col_inflamed, col_stricture = "Inflamed", "Stricture"  # in metadata file
col_sample_id_o, col_sample_id = "Sample ID", "Sample"  # in metadata, new
key_uninfl, key_infl, key_stric = "Uninflamed", "Inflamed", "Stricture"
m_d = (pd.read_excel if file_mdf[-4:] == "xlsx" else pd.read_csv)(
    file_mdf, dtype={"Slide ID": str}).rename({
        "Name": col_subject, "Inflammation": col_inflamed}, axis=1)
m_d.loc[:, col_condition] = m_d.apply(lambda x: "Stricture" if x[
    col_stricture].lower() in ["stricture", "yes"] else x[
        col_inflamed].capitalize(), axis=1)  # inflamation/stricture condition
m_d.loc[:, col_sample_id] = m_d[[col_condition, col_sample_id_o]].apply(
    "-".join, axis=1)

# Find File Path
files = functools.reduce(lambda i, j: i + j, [[os.path.join(
    run, i) for i in os.listdir(os.path.join(
        dir_data, run))] for run in os.listdir(dir_data)])
file_path = np.array(files)[np.where([libid == os.path.basename(
    x).split("__")[2].split("-")[0] for x in files])[0][0]]

# Load Data
lib = m_d.reset_index().set_index(col_sample_id_o).loc[libid][col_sample_id]
self = cr.Spatial(os.path.join(dir_data, file_path), library_id=lib)
self.update_from_h5ad(os.path.join(out_dir, lib + ".h5ad"))
self.get_layer("counts", inplace=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<<< INITIALIZING SPATIAL CLASS OBJECT >>>

INFO     reading                                                                                                   
         /mnt/cho_lab/bbdata2/outputs/TUQ97N/CHO-001/output-XETG00189__0010700__50452A-TUQ97N-EA__20240126__205019/
         cell_feature_matrix.h5                                                                                    


# Crop Data by Xenium Explorer Selection

In [ ]:
coord_suffix = "mucosa"
dir_coord = os.path.join(out_dir, "coordinates", coord_suffix)  # coordinates
sdata = self.crop(os.path.join(out_dir, libid + f"_{coord_suffix}.csv"))